In [1]:
%pip install --upgrade --quiet  azure-search-documents
%pip install --upgrade --quiet  azure-identity


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain_openai import AzureOpenAIEmbeddings, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

In [3]:
azure_endpoint: str = "https://demoopenairesource.openai.azure.com/"
azure_openai_api_key: str = "55FnF6s4J4OywbWkhBg75P6iFekRErJK3IYW40zlwD5EfUnFoH5MJQQJ99BBACHYHv6XJ3w3AAABACOGEtm9"
azure_openai_api_version: str = "2023-05-15"
azure_deployment: str = "text-embedding-3-small"

In [5]:
import os
os.getcwd()

'/workspaces/VoiceDemo'

In [6]:
# loader = PyPDFLoader("2WPolicy.pdf")
loader = PyPDFLoader("CustomerInfo.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(documents)

In [7]:
embeddings: AzureOpenAIEmbeddings = AzureOpenAIEmbeddings(
    azure_deployment=azure_deployment,
    openai_api_version=azure_openai_api_version,
    azure_endpoint=azure_endpoint,
    api_key=azure_openai_api_key,
)

vector_data = [
    {"id": str(i), "content": chunk.page_content, "vector": embeddings.embed_query(chunk.page_content)}
    for i, chunk in enumerate(docs)
]


In [8]:
len(vector_data[0]['vector'])

1536

In [9]:
vector_store_address: str = "https://testaisearchmctltfs030225.search.windows.net"
vector_store_password: str = "X5ayCxEXO3KNEKDgUp6hn466Gd7iDsukNQujhRfXV2AzSeDNKv8a"

In [10]:
index_name: str = "loan-documents-index"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [11]:
# # Specify additional properties for the Azure client such as the following https://github.com/Azure/azure-sdk-for-python/blob/main/sdk/core/azure-core/README.md#configurations
# vector_store: AzureSearch = AzureSearch(
#     azure_search_endpoint=vector_store_address,
#     azure_search_key=vector_store_password,
#     index_name=index_name,
#     embedding_function=embeddings.embed_query,
#     # Configure max retries for the Azure client
#     additional_search_client_options={"retry_total": 4},
# )

In [12]:
import json

# Convert documents to vector format
vector_data = [
    {
        "id": str(i),  # Unique ID for each document
        "content": doc.page_content,  # Original text for keyword search
        "vector": embeddings.embed_query(doc.page_content),  # Embedding vector
    }
    for i, doc in enumerate(docs)  # docs is your list of Document objects
]

# Now upload vector_data instead of docs
vector_store.client.upload_documents(documents=vector_data)

In [13]:
# vector_store.add_documents(documents=vector_data)